# Run Indexamajig with options for peakfinding, indexing and integration

In [ ]:
from gandalf_radial_iterator import gandalf_iterator

geomfile_path = "/home/bubl3932/files/LTA_sim/LTAsim.geom"      # .geom file
cellfile_path = "/home/bubl3932/files/LTA_sim/LTA.cell"         # .cell file
input_path =   "/home/bubl3932/files/LTA_sim/simulation-43"     # .h5 folder will also be output folder

output_file_base = "LTA"    # output files will be named output_file_base_xcoord_ycoord.h5

num_threads = 24            # number of threads to use
x, y = 512.5, 512.5         # default beam center
max_radius = 1              # maximum radius in pixels
step = 0.1                  # grid granularity in pixels

extra_flags=[
# PEAKFINDING
"--no-revalidate",
"--no-half-pixel-shift",
"--peaks=peakfinder9",
"--min-snr=1",
"--min-snr-biggest-pix=1",
"--min-sig=3",
"--local-bg-radius=10",
# INDEXING
"--indexing=xgandalf",
"--tolerance=10,10,10,5",
"--no-refine",
# "--no-retry",
"--xgandalf-sampling-pitch=5",
"--xgandalf-grad-desc-iterations=1",
"--xgandalf-tolerance=0.02",
# "--xgandalf-no-deviation-from-provided-cell",
# INTEGRATION
"--integration=rings",
"--int-radius=4,5,9",
"--fix-profile-radius=70000000",
# OUTPUT
"--no-non-hits-in-stream",
]

"""Examples of extra flags(see crystfel documentation https://www.desy.de/~twhite/crystfel/manual-indexamajig.html):"""

""" Basic options
"--highres=n",
"--no-image-data",
"""

""" Peakfinding
"--peaks=cxi",
"--peak-radius=inner,middle,outer",
"--min-peaks=n",
"--median-filter=n",
"--filter-noise",
"--no-revalidate",
"--no-half-pixel-shift",

"--peaks=peakfinder9",
"--min-snr=1",
"--min-snr-peak-pix=6",
"--min-snr-biggest-pix=1",
"--min-sig=9",
"--min-peak-over-neighbour=5",
"--local-bg-radius=5",

"--peaks=peakfinder8",
"--threshold=45",
"--min-snr=3",
"--min-pix-count=3",
"--max-pix-count=500",
"--local-bg-radius=9",
"--min-res=30",
"--max-res=500",
"""

""" Indexing
"--indexing=xgandalf",

"--tolerance=tol"
"--no-check-cell",
"--no-check-peaks",
"--multi",
"--no-retry",
"--no-refine",

"--xgandalf-sampling-pitch=n"
"--xgandalf-grad-desc-iterations=n"
"--xgandalf-tolerance=n"
"--xgandalf-no-deviation-from-provided-cell"
"--xgandalf-max-lattice-vector-length=n"
"--xgandalf-min-lattice-vector-length=n"
"--xgandalf-max-peaks=n"
"--xgandalf-fast-execution"
"""

""" Integration
"--fix-profile-radius=n",
"--fix-divergence=n",
"--integration=rings",
"--int-radius=4,5,10",
"--push-res=n",
"--overpredict",
"--cell-parameters-only",
"""

""" Output
"--no-non-hits-in-stream",
"--no-peaks-in-stream",
"--no-refls-in-stream",
"--serial-offset
"""

gandalf_iterator(x, y, geomfile_path, cellfile_path, input_path, output_file_base, num_threads, max_radius=max_radius, step=step, extra_flags=extra_flags)


# Evaluate the IQM with chosen weights for all frames and all index results

In [ ]:
from automate_evaluation import automate_evaluation

stream_file_folder = "/home/bubl3932/files/UOX1/UOX1_original_IQM_v3"   # Folder with stream file results from indexamajig. 
                                                                        # Note that all stream files in the folder will be processed.

weights_list = [
    (1, 1, 1, 1, 1, 1)
]

"""
Each weight corresponds to one of the six metrics used in calculating the combined IQM value.
The combined IQM is computed by first normalizing each metric across all stream files, then 
multiplying each normalized metric by its assigned weight, and finally summing the results.
The order (or keys) of the weights must match the following metrics:

- 'weighted_rmsd'
- 'fraction_outliers'
- 'length_deviation'
- 'angle_deviation'
- 'peak_ratio'
- 'percentage_indexed'

Multiple weight combinations can be specified if needed.
"""

automate_evaluation(stream_file_folder, weights_list)

# Merge the best results stream file

In [ ]:
from merge import merge

stream_file = f"{input_path}/{output_file_base}_-{x}_-{y}.stream"
pointgroup = "m-3m"
num_threads = 24
iterations = 5

output_dir = merge(
    stream_file,
    pointgroup=pointgroup,
    num_threads=num_threads,
    iterations=iterations,
)

if output_dir is not None:
    print("Merging done. Results are in:", output_dir)

# Convert to shelx compatible .hkl

In [ ]:

import os 
from convert_hkl_crystfel_to_shelx import convert_hkl_crystfel_to_shelx 
convert_hkl_crystfel_to_shelx(os.path.join(output_dir,"crystfel.hkl"))

# Convert to mtz

In [ ]:
from convert_hkl_to_mtz import convert_hkl_to_mtz
cellfile_path = "/home/bubl3932/files/LTA_sim/LTA.cell"
convert_hkl_to_mtz(output_dir,cellfile_path=cellfile_path)